In [0]:
import pandas as pd
import numpy as np

# from google.colab import files
# uploaded = files.upload()

In [0]:
import io
df1 = pd.read_csv('/content/AIAGE_search.csv',sep=';')
df2 = pd.read_csv('/content/ERA_search.csv',sep=',')
df3 = pd.read_csv('/content/PropNex_search.csv',sep=',')
df4 = pd.read_csv('/content/IT 2019-10-18 16_47_25.csv',sep=',')
df5 = pd.read_csv('/content/Marketing&Advertising.csv',sep=',')
df6 = pd.read_csv('/content/contacts 2019-10-18 15_58_25.csv',sep=',')

df = pd.concat([df1, df2, df3,df4,df5,df6])

In [3]:
df.drop_duplicates(inplace=True)
df.head()

,id,Full name,Email,Profile url,First name,Last name,Title,Avatar,Location,Address,Birthday,Summary,Twitter,Phone 1,Phone 1 type,Phone 2,Phone 2 type,Phone 3,Phone 3 type,Messenger 1,Messenger 1 type,Messenger 2,Messenger 2 type,Messenger 3,Messenger 3 type,Website 1,Website 2,Website 3,Organization 1,Organization Title 1,Organization Start 1,Organization End 1,Organization Description 1,Organization Location 1,Organization LI URL 1,Organization LI ID 1,Organization WWW 1,Organization Domain 1,Organization 2,Organization Title 2,...,Organization Title 7,Organization Start 7,Organization End 7,Organization Description 7,Organization Location 7,Organization LI URL 7,Organization LI ID 7,Organization WWW 7,Organization Domain 7,Education 1,Education Degree 1,Education FOS 1,Education Grade 1,Education Start 1,Education End 1,Education Description 1,Education 2,Education Degree 2,Education FOS 2,Education Grade 2,Education Start 2,Education End 2,Education Description 2,Education 3,Education Degree 3,Education FOS 3,Education Grade 3,Education Start 3,Education End 3,Education Description 3,Skills,Followers,Relationship,Connected at,Industry,Mutual Count,Mutual,Mutual 1,Mutual 2,Interests
0,weienthong,Wei En THONG,NaN,https://www.linkedin.com/in/weienthong/,Wei,Thong,Group Governance and Operational Risk Manageme...,https://media.licdn.com/dms/image/C5103AQGGqm3...,Singapore,NaN,March 8,If there is one element of myself that I have ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GREAT EASTERN,Group Governance and Operational Risk Manageme...,May 2019,PRESENT,Devised new risk culture transformation framew...,Singapore,https://www.linkedin.com/company/15626,15626,NaN,NaN,Singapore Management University,Teaching Assistant - International Macroeconomics,...,Training Specialist,Oct 2014,Dec 2015,Planned key leadership courses and professiona...,Singapore,https://www.linkedin.com/search/results/index/...,NaN,NaN,NaN,Singapore Management University,Bachelor of Science - BS,Economics,NaN,2016.0,2019,SMU Scholars' ProgrammeDean's List (Academic Y...,Hwa Chong Institution,A Level,NaN,NaN,2008.0,2013,"Vice President, HCI Japanese Cultural Club (20...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Public Speaking : 9, Market Research : 9, Qua...",1031.0,2,NaN,Financial Services,19.0,"Daniel Farhan, Kelvin Wong",Daniel Farhan,Kelvin Wong,HSBC; Jeff Weiner; Accenture; UBS; Tony Robbin...
1,amberpoo,"Poo Men Shuen, Amber",NaN,https://www.linkedin.com/in/amberpoo/,Poo,Shuen,Estate Analyst at Bequest Pte Ltd,https://media.licdn.com/dms/image/C4E03AQFPcbS...,Singapore,NaN,NaN,"Known to be a great team player, Amber has vol...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GREAT EASTERN,Life Insurance Specialist,May 2017,PRESENT,NaN,Singapore,https://www.linkedin.com/company/15626,15626,NaN,NaN,Bequest Pte Ltd,Estate Analyst,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Singapore Management University,Bachelor of Science (Economics),Economics,NaN,2013.0,2017,NaN,Jurong Junior College,'A' Levels,Science,NaN,2011.0,2012,NaN,Nan Hua High School,'O' Levels,NaN,NaN,2007.0,2010,NaN,"""Microsoft Office : 11, Microsoft Excel : 10, ...",NaN,2,NaN,Financial Services,19.0,"Kelvin Wong, Faustina Helena",Kelvin Wong,Faustina Helena,Daniel Goleman; IBM; Ray Dalio; Satya Nadella;...
2,jerome-lim,Jerome Lim,NaN,https://www.linkedin.com/in/jerome-lim/,Jerome,Lim,Impacting Life One at a Time,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",Singapore,NaN,NaN,"In life, we are facing with a lot of uncertain...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GREAT EASTERN,Financial Advisor,Nov 2016,PRESENT,Also as a Life Planner with Calibre Advisory (...,Singapore,https://www.linkedin.com/company/15626,15626,NaN,NaN,Singapore Chamber of Commerce and Industry / S...,Committee Member,...,Sales Engineer,Jul 2007,Jun 2011,NaN,Singapore,https://www.linkedin.com/search/results/index/...,NaN,NaN,NaN,Northumbria University,Bachelor's degree,

In [4]:
print("Before dropping:",len(df.columns),"columns")
#drop columns with more than 60% of values missing
df.dropna(thresh = (len(df) * .4) , axis = 1, inplace = True)
#Dont drop organization columns
print("After dropping:",len(df.columns),"columns")

Before dropping: 129 columns
After dropping: 64 columns


# Spending power

In [5]:
#Get student status

df['Education End 1'][df['Education End 1'] == "PRESENT"] = 2023 #Just assume grduation date is 2023 if current student +4 years
df['Education End 1'] = df['Education End 1'].fillna(0) #Assume if no values entered, they are not student
df['Student_Status'] = (df['Education End 1'].astype(int) < 2019).astype(int)
# df['Student_status'].head()
df.head()                      

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,Full name,Profile url,First name,Last name,Title,Avatar,Location,Summary,Organization 1,Organization Title 1,Organization Start 1,Organization End 1,Organization Description 1,Organization Location 1,Organization LI URL 1,Organization LI ID 1,Organization 2,Organization Title 2,Organization Start 2,Organization End 2,Organization Description 2,Organization Location 2,Organization LI URL 2,Organization LI ID 2,Organization 3,Organization Title 3,Organization Start 3,Organization End 3,Organization Description 3,Organization Location 3,Organization LI URL 3,Organization LI ID 3,Organization 4,Organization Title 4,Organization Start 4,Organization End 4,Organization Description 4,Organization Location 4,Organization LI URL 4,Organization LI ID 4,Organization 5,Organization Title 5,Organization Start 5,Organization End 5,Organization LI URL 5,Education 1,Education Degree 1,Education FOS 1,Education Start 1,Education End 1,Education 2,Education Degree 2,Education FOS 2,Education Start 2,Education End 2,Skills,Relationship,Industry,Mutual Count,Mutual,Mutual 1,Mutual 2,Interests,Student_Status
0,weienthong,Wei En THONG,https://www.linkedin.com/in/weienthong/,Wei,Thong,Group Governance and Operational Risk Manageme...,https://media.licdn.com/dms/image/C5103AQGGqm3...,Singapore,If there is one element of myself that I have ...,GREAT EASTERN,Group Governance and Operational Risk Manageme...,May 2019,PRESENT,Devised new risk culture transformation framew...,Singapore,https://www.linkedin.com/company/15626,15626,Singapore Management University,Teaching Assistant - International Macroeconomics,Aug 2018,Dec 2018,Improving students' understanding of internati...,Singapore,https://www.linkedin.com/company/165981,165981,UBS,Group Finance Summer Intern,May 2018,Jul 2018,Upheld strong risk management culture through ...,Singapore,https://www.linkedin.com/company/1214,1214,Prudential Assurance Company Singapore,Financial Consultant,May 2017,May 2018,Helped clients successfully kick-start their f...,Singapore,https://www.linkedin.com/company/2140469,2.14047e+06,Singapore Management University,Teaching Assistant - Intermediate Maths for Ec...,Aug 2017,Nov 2017,https://www.linkedin.com/company/165981,Singapore Management University,Bachelor of Science - BS,Economics,2016.0,2019,Hwa Chong Institution,A Level,NaN,2008.0,2013,"""Public Speaking : 9, Market Research : 9, Qua...",2,Financial Services,19.0,"Daniel Farhan, Kelvin Wong",Daniel Farhan,Kelvin Wong,HSBC; Jeff Weiner; Accenture; UBS; Tony Robbin...,0
1,amberpoo,"Poo Men Shuen, Amber",https://www.linkedin.com/in/amberpoo/,Poo,Shuen,Estate Analyst at Bequest Pte Ltd,https://media.licdn.com/dms/image/C4E03AQFPcbS...,Singapore,"Known to be a great team player, Amber has vol...",GREAT EASTERN,Life Insurance Specialist,May 2017,PRESENT,NaN,Singapore,https://www.linkedin.com/company/15626,15626,Bequest Pte Ltd,Estate Analyst,May 2017,PRESENT,NaN,269a new bridge road,https://www.linkedin.com/company/3783566,3.78357e+06,Equinix EMEA,SSC and Project Intern,Apr 2016,Dec 2016,-Reconciled statement of accounts and improved...,Singapore,https://www.linkedin.com/company/200342,200342,Citi,Temporary Bank Officer at Assets Operations,Jul 2015,Sep 2015,-Pioneered and represented the bank in its Kno...,Singapore,https://www.linkedin.com/company/11448,11448,Garden picks Food Manufacturing LLP,Customer Service cum Administrative Officer,May 2014,Aug 2014,https://www.linkedin.com/search/results/index/...,Singapore Management University,Bachelor of Science (Economics),Economics,2013.0,2017,Jurong Junior College,'A' Levels,Science,2011.0,2012,"""Microsoft Office : 11, Microsoft Excel : 10, ...",2,Financial Services,19.0,"Kelvin Wong, Faustina Helena",Kelvin Wong,Faustina Helena,Daniel Goleman; IBM; Ray Dalio; Satya Nadella;...,1
2,jerome-lim,Jerome Lim,https://www.linkedin.com/in/jerome-lim/,Jerome,Lim,Impacting Life One at a Time,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",Singapore,"In life, we are faci

In [6]:
#!pip install fuzzywuzzy

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#Get highest education

data = {'School':[4,3,2,1],
        'School1':['University','Polytechnic','Junior College','Below tertiary'],
        'School2':['College','High School','','Primary']}

df_match_edu = pd.DataFrame(data)


match = list(map(lambda x: process.extractOne(x, df_match_edu['School1'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Education 1'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore1'] = list(match['score'])
df['edu1'] = list(df_match_edu.loc[match['index']].School)

match = list(map(lambda x: process.extractOne(x, df_match_edu['School2'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Education 1'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore2'] = list(match['score'])
df['edu2'] = list(df_match_edu.loc[match['index']].School)



df['eduscore'] = df[['eduscore1', 'eduscore2']].max(axis=1)
df['Education_category'] = np.where(df['eduscore1']>= df['eduscore2'], df['edu1'], df['edu2'])


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
#Get years of experience by subtracting last worked time with the start of their first job 
df['Last_worked_year'] = np.NaN
df['Organization End 1'][df['Organization End 1'] == "PRESENT"] = "2019"
df['Organization End 1'] = df['Organization End 1'].str.extract('(\d+)')

df['Last_worked_year'] = df['Organization End 1']


df['First_worked_year'] = np.NaN

df['Organization Start 1'] = df['Organization Start 1'].str.extract('(\d+)')
df['Organization Start 2'] = df['Organization Start 2'].str.extract('(\d+)')
df['Organization Start 3'] = df['Organization Start 3'].str.extract('(\d+)')
df['Organization Start 4'] = df['Organization Start 4'].str.extract('(\d+)')
df['Organization Start 5'] = df['Organization Start 5'].str.extract('(\d+)')

df['First_worked_year'][df['Organization Start 1'].notnull()] = df['Organization Start 1'][df['Organization Start 1'].notnull()]
df['First_worked_year'][df['Organization Start 2'].notnull()] = df['Organization Start 2'][df['Organization Start 2'].notnull()]
df['First_worked_year'][df['Organization Start 3'].notnull()] = df['Organization Start 3'][df['Organization Start 3'].notnull()]
df['First_worked_year'][df['Organization Start 4'].notnull()] = df['Organization Start 4'][df['Organization Start 4'].notnull()]
df['First_worked_year'][df['Organization Start 5'].notnull()] = df['Organization Start 5'][df['Organization Start 5'].notnull()]

df['Last_worked_year'] = df['Last_worked_year'].fillna(0).str.extract('(\d+)')
df['First_worked_year'] = df['First_worked_year'].fillna(0).str.extract('(\d+)')

df['Working_years'] = df['Last_worked_year'].astype(float) - df['First_worked_year'].astype(float) 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [0]:
# 2) Job title for spending power.

data = {'Title':[3,2,1],
        'Title1':['Director','Manager','Associate'],
        'Title2':['Partner','Supervisor','Executive']}

df_match_edu = pd.DataFrame(data)


match = list(map(lambda x: process.extractOne(x, df_match_edu['Title1'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Title'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore1'] = list(match['score'])
df['edu1'] = list(df_match_edu.loc[match['index']].Title)

match = list(map(lambda x: process.extractOne(x, df_match_edu['Title2'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Title'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore2'] = list(match['score'])
df['edu2'] = list(df_match_edu.loc[match['index']].Title)


df['Title_score'] = df[['eduscore1', 'eduscore2']].max(axis=1)
df['Title'] = np.where(df['eduscore1']>= df['eduscore2'], df['edu1'], df['edu2'])


In [0]:
# Size of company: Bigger means higher chance to get a corporate scheme/ car funds, as well as receive better remuneration

data = {'Title':[1,2,3],
        'Title1':['Munch Technologies','Shopee','AIA'],
        'Title2':['Ant Financial','Lazada','Prudential'],
        'Title3':['SG Interactive','Advisors Clique','Great Eastern']}

df_match_edu = pd.DataFrame(data)

match = list(map(lambda x: process.extractOne(x, df_match_edu['Title1'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Organization 1'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore1'] = list(match['score'])
df['edu1'] = list(df_match_edu.loc[match['index']].Title)

match = list(map(lambda x: process.extractOne(x, df_match_edu['Title2'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Organization 1'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore2'] = list(match['score'])
df['edu2'] = list(df_match_edu.loc[match['index']].Title)

match = list(map(lambda x: process.extractOne(x, df_match_edu['Title3'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Organization 1'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore3'] = list(match['score'])
df['edu3'] = list(df_match_edu.loc[match['index']].Title)

df['eduscore'] = df[['eduscore1', 'eduscore2']].max(axis=1)
df['edu'] = np.where(df['eduscore1']>= df['eduscore2'], df['edu1'], df['edu2'])
df['Companysize'] = np.where(df['eduscore']>= df['eduscore3'], df['edu'], df['edu3'])
df['Companysize_score'] = df[['eduscore', 'eduscore3']].max(axis=1)



# Leasing Propensity

In [0]:
#  Job scope

data = {'Title':[1,2,3],
        'Title1':['Admin','Director','Advisor'],
        'Title2':['Executive','Partner','Agent']}

df_match_edu = pd.DataFrame(data)


match = list(map(lambda x: process.extractOne(x, df_match_edu['Title1'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Title'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore1'] = list(match['score'])
df['edu1'] = list(df_match_edu.loc[match['index']].Title)

match = list(map(lambda x: process.extractOne(x, df_match_edu['Title2'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Title'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore2'] = list(match['score'])
df['edu2'] = list(df_match_edu.loc[match['index']].Title)


df['Scope_score'] = df[['eduscore1', 'eduscore2']].max(axis=1)
df['Scope'] = np.where(df['eduscore1']>= df['eduscore2'], df['edu1'], df['edu2'])

In [0]:
# predict family by age. assume higher working experience means have family. 

df['Family'] = (df['Working_years'] >= 7).astype(int)

In [0]:
# type of industry: customer facing or non-customer facing

data = {'Title':[1,0],
        'Title1':['Financial/insurance Services','Accounting'],
        'Title2':['Real Estate','Information Technology']}

df_match_edu = pd.DataFrame(data)

match = list(map(lambda x: process.extractOne(x, df_match_edu['Title1'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Industry'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore1'] = list(match['score'])
df['edu1'] = list(df_match_edu.loc[match['index']].Title)

match = list(map(lambda x: process.extractOne(x, df_match_edu['Title2'], scorer=fuzz.token_set_ratio, processor=lambda x: x),df['Industry'].astype(str).str.strip()))
match = pd.DataFrame(match)
match.columns = ['match', 'score', 'index']
df['eduscore2'] = list(match['score'])
df['edu2'] = list(df_match_edu.loc[match['index']].Title)


df['Industry_score'] = df[['eduscore1', 'eduscore2']].max(axis=1)
df['Industry'] = np.where(df['eduscore1']>= df['eduscore2'], df['edu1'], df['edu2'])

In [14]:
#mentioned the keywords of meeting, clients, connecting, plan , insurance,travel, sales in their profile

keywords = ['meeting','clients','connecting','plan','insurance','travel']

import gensim.downloader as api
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('/content/glove.6B.50d.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
df['Summary'].fillna("",inplace=True)
df['Profile_similarity'] = np.NaN

prof_sim = []
for sent in df['Summary']:
  average_similarity = 0
  word_count = 1
  avg_sim = 0
  for word in sent:
    max_sim = 0
    word_count += 1
    for kword in keywords:
      if word in model.vocab and kword in model.vocab:
        if model.similarity(word,kword) > max_sim:
          max_sim = model.similarity(word,kword)
    
    avg_sim += max_sim
  
  avg_sim /= word_count
  prof_sim.append(avg_sim)  

df['Profile_similarity'] =  pd.Series(prof_sim)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [16]:

df_final = df[['id','Full name','Profile url','Location','Title','Companysize','Working_years','Student_Status','Education_category','Scope','Industry','Family','Profile_similarity']]

df_final.head()

,id,Full name,Profile url,Location,Title,Companysize,Working_years,Student_Status,Education_category,Scope,Industry,Family,Profile_similarity
0,weienthong,Wei En THONG,https://www.linkedin.com/in/weienthong/,Singapore,2,3,2.0,0,4,1,1,0,0.320495
1,amberpoo,"Poo Men Shuen, Amber",https://www.linkedin.com/in/amberpoo/,Singapore,1,3,5.0,1,4,1,1,0,0.339094
2,jerome-lim,Jerome Lim,https://www.linkedin.com/in/jerome-lim/,Singapore,3,3,5.0,1,4,1,1,0,0.333927
3,amanda-lim-30813254,Amanda Lim,https://www.linkedin.com/in/amanda-lim-30813254/,Singapore,2,3,9.0,1,4,1,1,1,0.000000
4,ijeremytang,Jeremy Tang,https://www.linkedin.com/in/ijeremytang/,Singapore,2,3,2.0,0,3,1,1,0,0.322602


In [17]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()

df_final[['Title','Companysize','Working_years','Student_Status','Education_category','Scope','Industry','Family','Profile_similarity']] = mm_scaler.fit_transform(df_final[['Title','Companysize','Working_years','Student_Status','Education_category','Scope','Industry','Family','Profile_similarity']])
df_final.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,id,Full name,Profile url,Location,Title,Companysize,Working_years,Student_Status,Education_category,Scope,Industry,Family,Profile_similarity
0,weienthong,Wei En THONG,https://www.linkedin.com/in/weienthong/,Singapore,0.5,1.0,0.050,0.0,1.000000,0.0,1.0,0.0,0.883665
1,amberpoo,"Poo Men Shuen, Amber",https://www.linkedin.com/in/amberpoo/,Singapore,0.0,1.0,0.125,1.0,1.000000,0.0,1.0,0.0,0.934944
2,jerome-lim,Jerome Lim,https://www.linkedin.com/in/jerome-lim/,Singapore,1.0,1.0,0.125,1.0,1.000000,0.0,1.0,0.0,0.920697
3,amanda-lim-30813254,Amanda Lim,https://www.linkedin.com/in/amanda-lim-30813254/,Singapore,0.5,1.0,0.225,1.0,1.000000,0.0,1.0,1.0,0.000000
4,ijeremytang,Jeremy Tang,https://www.linkedin.com/in/ijeremytang/,Singapore,0.5,1.0,0.050,0.0,0.666667,0.0,1.0,0.0,0.889472


In [0]:
df_final.to_csv('linkedin_processed.csv')